## **Setup**

In [1]:
# Import the necesary packages
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [2]:
config = {
    # Basic information
    "AUTHOR": "Kiernan",
    
    # Training params
    "LR": 0.001,
    "BATCH_SIZE": 32,
    "EPOCHS": 30,
    
    # Model params
    "FIRST_FILTERS": 16,
    "CONV_LAYERS": 4,
    "N_FILTERS": 8,
    "KERNEL_SIZE": (3,3),
    "EMBEDDING_SIZE": 16,
    "DROPOUT": 0.2
}

## **Initialize WANDB**

In [3]:
import wandb
from wandb.keras import WandbCallback
from secrets import WANDB
wandb.login(key=WANDB)
run = wandb.init(project="deep-clustering-evaluation", entity="kmcguigan", group="base-model", config=config, job_type="train")

wandb: Currently logged in as: all-off-nothing (use `wandb login --relogin` to force relogin)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\kiern/.netrc


## **Loading Data**

In [4]:
(X, y), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

X = (X.astype(np.float32) - 127.5) / 127.5
X_test = (X_test.astype(np.float32) - 127.5) / 127.5

X = X.reshape((*X.shape, 1))
X_test = X_test.reshape((*X_test.shape, 1))

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=X_test.shape[0], shuffle=True)
print(f"Train data shape: {X_train.shape} Val data shape: {X_val.shape} Test data shape: {X_test.shape}")

Train data shape: (50000, 28, 28, 1) Val data shape: (10000, 28, 28, 1) Test data shape: (10000, 28, 28, 1)


## **Create Model**

In [5]:
def create_body(image_shape):
    inputs = tf.keras.layers.Input(shape=image_shape)
    
    def conv_block(layer_inputs, n_filters, kernel_size, **kwargs):
        x = tf.keras.layers.Conv2D(n_filters, kernel_size, padding="same", **kwargs)(layer_inputs)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.ReLU()(x)
        return x
    
    x = conv_block(inputs, config["FIRST_FILTERS"], config["KERNEL_SIZE"], strides=2)
    for _ in range(config["CONV_LAYERS"]):
        x = conv_block(x, config["N_FILTERS"], config["KERNEL_SIZE"])
    
    x = tf.keras.layers.Conv2D(config["EMBEDDING_SIZE"], (1,1), padding="same")(x)
    outputs = tf.keras.layers.GlobalAveragePooling2D()(x)
    return tf.keras.models.Model(inputs=inputs, outputs=outputs, name="body")

body = create_body(X_train.shape[1:])
body.summary()

Model: "body"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 14, 14, 16)        160       
                                                                 
 batch_normalization (BatchN  (None, 14, 14, 16)       64        
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 14, 14, 16)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 8)         1160      
                                                                 
 batch_normalization_1 (Batc  (None, 14, 14, 8)        32        
 hNormalization)                                              

In [6]:
def create_head(n_classes):
    inputs = tf.keras.layers.Input(shape=(config["EMBEDDING_SIZE"]))
    x = tf.keras.layers.Dropout(config["DROPOUT"])(inputs)
    outputs = tf.keras.layers.Dense(n_classes, activation='softmax')(x)
    return tf.keras.models.Model(inputs=inputs, outputs=outputs, name="head")

head = create_head(10)
head.summary()

Model: "head"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 16)]              0         
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense (Dense)               (None, 10)                170       
                                                                 
Total params: 170
Trainable params: 170
Non-trainable params: 0
_________________________________________________________________


In [7]:
model = tf.keras.models.Sequential([
    body,
    head
],
    name="combinedModel"
)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
optimizer = tf.keras.optimizers.Adam(learning_rate=config['LR'])
metrics = [tf.keras.metrics.SparseCategoricalAccuracy(),]
model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
model.summary()

Model: "combinedModel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 body (Functional)           (None, 16)                3408      
                                                                 
 head (Functional)           (None, 10)                170       
                                                                 
Total params: 3,578
Trainable params: 3,482
Non-trainable params: 96
_________________________________________________________________


## **Training**

In [ ]:
stopper = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)
lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2)

hist = model.fit(X_train, y_train,
                 validation_data=(X_val, y_val),
                 batch_size=config["BATCH_SIZE"],
                 validation_batch_size=config["BATCH_SIZE"],
                 epochs=config["EPOCHS"],
                 callbacks=[stopper, lr_reducer, WandbCallback(predictions=8, input_type='images', validation_data=(X_val, y_val))])

model.save(os.path.join(run.dir, "model.h5"))

Epoch 1/30
1563/1563 [==============================] - 29s 18ms/step - loss: 0.9502 - sparse_categorical_accuracy: 0.6616 - val_loss: 0.8802 - val_sparse_categorical_accuracy: 0.6524 - lr: 0.0010
Epoch 2/30
1563/1563 [==============================] - 27s 17ms/step - loss: 0.3274 - sparse_categorical_accuracy: 0.9077 - val_loss: 0.3435 - val_sparse_categorical_accuracy: 0.8950 - lr: 0.0010
Epoch 3/30
1563/1563 [==============================] - 27s 17ms/step - loss: 0.2250 - sparse_categorical_accuracy: 0.9340 - val_loss: 0.2887 - val_sparse_categorical_accuracy: 0.9060 - lr: 0.0010
Epoch 4/30
1563/1563 [==============================] - 26s 17ms/step - loss: 0.1872 - sparse_categorical_accuracy: 0.9456 - val_loss: 0.1886 - val_sparse_categorical_accuracy: 0.9399 - lr: 0.0010
Epoch 5/30
1563/1563 [==============================] - 25s 16ms/step - loss: 0.1644 - sparse_categorical_accuracy: 0.9521 - val_loss: 0.2081 - val_sparse_categorical_accuracy: 0.9388 - lr: 0.0010
Epoch 6/30
1563

In [ ]:
fig = plt.figure(figsize=(12,12), tight_layout=True)

plt.subplot(311)
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc='upper right')

plt.subplot(312)
plt.plot(hist.history['sparse_categorical_accuracy'])
plt.plot(hist.history['val_sparse_categorical_accuracy'])
plt.title('Model Categorical Accuracy')
plt.ylabel('Categorical Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc='upper left')

plt.subplot(313)
plt.plot(hist.history['lr'])
plt.title('Learining Rate')
plt.ylabel('Learning Rate')
plt.xlabel('Epoch')

run.log({'train_graph': fig})
plt.show()

In [ ]:
ev = model.evaluate(X_test, y_test)
run.log({'test/loss':ev[0], 'test/categorical_accuracy':ev[1]})

## **Analyze Test Results**

In [ ]:
preds = model.predict(X_test)

for i in range(16):
    run.log({f"test/image{i}": wandb.Image(X_test[i,:,:,:], caption=f"Pred: {preds[i]} Actual: {y_test[i]}")})

cf = confusion_matrix(y_test, np.argmax(preds, axis=-1))
cf

In [ ]:
class_names = [x for x in range(0, 10, 1)]
fig = plt.figure(figsize=(12,12))
sns.heatmap(cf, annot=True, xticklabels=class_names, yticklabels=class_names, cmap='Blues', robust=True)
run.log({'confusion_matrix': fig})
plt.show()

In [ ]:
run.finish()